## Instalations

In [3]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

## Imports

In [1]:
import ipywidgets as widgets
import io
from ipywidgets import interact,Layout
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import clear_output
from random import sample
from pymove import MoveDataFrame
from pymove import filters
from io import StringIO
from pymove.visualization import folium as f, matplotlib as mpl
import folium
from copy import copy
from os import listdir
%matplotlib inline

## Functions

In [9]:
import sys
import numpy as np

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def print_top_mem_vars(variables):
    """Usage: 
    mem.print_top_mem_vars(locals())
    mem.print_top_mem_vars(globals())
    """
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in variables.items()), key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

def reduce_mem_usage_automatic(df):
    """Reduces the memory usage of the given dataframe.
    Parameter
    ---------
    df : dataframe
        The input data to which the operation of memory reduction will be performed.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type) == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df[col] = df[col].astype(np.uint8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df[col] = df[col].astype(np.uint16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df[col] = df[col].astype(np.uint32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
            elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                df[col] = df[col].astype(np.uint64)
        elif col_type == np.float:
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

In [ ]:
#print_top_mem_vars(locals())

In [2]:
def setup_ui(df):
    
    out = widgets.Output()
    with out:
        display(df)
    return out

In [3]:
#@title **Demo Application**

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()

tab = widgets.Tab([out1, out2, out3, out4], layout=widgets.Layout(width="80%"))

tab.set_title(0, 'Trajectory Data Upload')
tab.set_title(1, 'View Trajectory Data')
tab.set_title(2, 'Embedding Upload')
tab.set_title(3, 'Intrinsic Evaluation')
tab.se

display(tab)

with out1:

    def func1(b):

        with df1:
            
            global Datasets
            
            df1.clear_output()
            new_values = b.new
            keys = list(new_values.keys())

            for k in keys:

                data = StringIO(str(new_values[k]["content"],'utf-8'))
                df=pd.read_csv(data,index_col=0)
                df.to_csv("Datasets/"+k)
                
            Datasets =  [dataframe.split(".")[0] for dataframe in listdir("Datasets/")]
            
            with text3_out:
                
                text3_out.clear_output()
                text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=Datasets)
                text3.observe(on_change_dataset, names='value')
                display(text3)
            
            display(pd.DataFrame({"Trajectory Data":Datasets}))
            
      
    
    df1 = setup_ui(pd.DataFrame({"Trajectory Data":[dataframe.split(".")[0] for dataframe in listdir("Datasets/")]}))
    b1 = widgets.FileUpload(acept=".csv",multiple=True,layout=widgets.Layout(width="20%"))
    b1.observe(func1,names="value")

    spc = widgets.Label("")
    spc1 = widgets.Label("",layout=widgets.Layout(width="9%"))
    spc2 = widgets.Label("",layout=widgets.Layout(width="5%"))
    h1_2 = widgets.HBox([spc2,df1])
    h1 = widgets.HBox(children=[widgets.Label(""),b1])
    h2 = widgets.VBox(children=[spc,h1,spc1,h1_2])

    display(h2)




with out2:
    
    global Datasets

    out_df_2 = setup_ui(widgets.Label(""))
    out_plot_2 = setup_ui(widgets.Label(""))
    out_df_2.layout = widgets.Layout(width = "300px",border='solid 1.5px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
    out_plot_2.layout = widgets.Layout(border='solid 2.0px',margin='0px 10px 10px 10px',padding='5px 5px 5px 5px')
  
    out_statistics = widgets.Output()
    out_sampling = widgets.Output()

    out_statistics.layout = widgets.Layout(border='solid 1.5px white',margin='10px 70px 10px 0px',
                                   padding='5px 5px 5px 5px')
  
    out_sampling.layout = widgets.Layout(border='solid 1.5px white',margin='20px 10px 10px 0px',
                                   padding='5px 5px 5px 5px')

    DatasetsList = [dataframe.split(".")[0] for dataframe in listdir("Datasets/")]               
    dataset_2  = DatasetsList[0]
    sampling = 10

    def Show_Statistics(b):

        with out_df_2:

            global dataset_2
            DF = pd.read_csv("Datasets/"+dataset_2+".csv",index_col=0)

            if(len(DF) > 0):
            
                out_df_2.clear_output()
                number_of_trajectories = len(set(DF["trajectory_id"]))
                length_trajectories = [len(DF[DF["trajectory_id"] == tj]) for tj in set(DF["trajectory_id"])]
                maximum_length_of_trajectories = max(length_trajectories)
                minimum_length_of_trajectories = min(length_trajectories)
                average_length_of_trajectories = int(sum(length_trajectories)/len(length_trajectories))

                df1 = ""
        
                if("location_label" in list(DF.columns)):
                    number_of_distinct_locations = len(set(DF["location_label"]))
                    df1 = pd.DataFrame({" ":[number_of_trajectories,maximum_length_of_trajectories,minimum_length_of_trajectories,average_length_of_trajectories,
                                  number_of_distinct_locations]}, index = ["number_of_trajectories","maximum_length_of_trajectories","minimum_length_of_trajectories",
                                                                          "average_length_of_trajectories","number_of_distinct_locations"])
                else:
                    df1 = pd.DataFrame({" ":[number_of_trajectories,maximum_length_of_trajectories,minimum_length_of_trajectories,average_length_of_trajectories]}, 
                                     index = ["number_of_trajectories","maximum_length_of_trajectories","minimum_length_of_trajectories",
                                                                          "average_length_of_trajectories"])


                display(df1)

            else:
                out_df_2.clear_output()
                display(widgets.Label("Dataset Vazio !"))

    def Plot_Sampling(b):

        with out_plot_2:

            global sampling
            DF = pd.read_csv("Datasets/"+dataset_2+".csv",index_col=0)
        

            if(len(DF) > 0):

                out_plot_2.clear_output()
                set_traj = set(DF["trajectory_id"])

                # Setar o Trjaectory_ID pra set o index
                # Setar no pymove, Filter byID

                Sampling_Traj = sample(set_traj,int(sampling))
                DataFrame = pd.concat([filters.by_label(DF, value = tj, label_name = "trajectory_id") for tj in Sampling_Traj],axis=0)
        

                move_df = MoveDataFrame(data=DataFrame, latitude="lat", longitude="lon", datetime="time", 
                                        traj_id='trajectory_id')
                figure = folium.Figure(width=800, height=300) 

                display(f.plot_trajectories(move_df).add_to(figure))

            else:

                out_plot_2.clear_output()
                display(widgets.Label("Dataset Vazio !"))

  
    def on_change_dataset(change):

        global dataset_2
        dataset_2 = change.new

    def on_change_sampling(change):

        global sampling
        sampling = change.new

    with out_statistics:
        
        text2 = widgets.Label("Trajectory Data")
        spc = widgets.Label("")
        text3 = widgets.Dropdown(description='',layout=widgets.Layout(width="80px"),options=DatasetsList)
        text3.observe(on_change_dataset, names='value')
        text3_out = setup_ui(text3)
        b3 = widgets.Button(description="Show Statistics", layout=widgets.Layout(width="120px"))
        b3.on_click(Show_Statistics)
        b3.style.button_color = "lightgray"

        h4 = widgets.VBox(children=[text2,widgets.HBox(children=[spc,text3_out,spc,b3])])
        h5 = widgets.VBox(children=[h4,widgets.Label("",layout=widgets.Layout(width="19%")),out_df_2])

        display(h5) 

    with out_sampling: 
        text_smp = widgets.Label("Sampling")
        text10 = widgets.Text(description='',layout=widgets.widgets.Layout(width="80px"))
        text10.observe(on_change_sampling, names = 'value')
        b4 = widgets.Button(description="Plot Trajectories",layout=widgets.Layout(width="120px"))
        b4.style.button_color = "lightgray"
        b4.on_click(Plot_Sampling)
        h8 = widgets.VBox(children=[text_smp,widgets.HBox(children=[text10,b4])])

        h9 = widgets.VBox(children=[h8,spc,out_plot_2])

        display(h9)

    h10 = widgets.HBox(children=[out_statistics,out_sampling])

    display(h10)
  
  
with out3:

    text = widgets.Label("Embedding")
    b1 = widgets.Button(description="Import", layout=widgets.Layout(width="30%"))
    b1.style.button_color = "lightgray"

    file = open("datasets2.png", "rb")
    image = file.read()
    img = widgets.Image(value=image,format='png',width=200,height=300)

    file = open("datasets4.png", "rb")
    image = file.read()
    img2 = widgets.Image(value=image,format='png',width=65,height=20)

    html = "<h1 style='font-size:15px'> Trajectory Data </h1>"
    h1_3 = widgets.VBox([widgets.HTML(value=html),img])

    html2 = "<h1 style='font-size:15px'> Embedding Model</h1>"
    h1_4 = widgets.VBox([widgets.HTML(value=html2),img2])

    spc = widgets.Label("")
    spc1 = widgets.Label("",layout=widgets.Layout(width="19%"))
    spc2 = widgets.Label("",layout=widgets.Layout(width="15%"))
    h1_2 = widgets.HBox([spc2,widgets.HBox([h1_4,spc,spc,spc,h1_3])])
    h1 = widgets.HBox(children=[text,widgets.Label(""),b1])
    h2 = widgets.VBox(children=[spc,widgets.VBox([h1,spc1,h1_2])])

    display(h2)


with out4:

    l1 = widgets.Label("Trajectory Data")
    l2 = widgets.Label("Embedding A")
    l3 = widgets.Label("Embedding B")
    spc = widgets.Label("")
    text14 = widgets.Dropdown(description='',layout=widgets.Layout(width="35%"),options=["External Sensors Trajectories","GeoLife GPS Trajectories"])
    text15 = widgets.Dropdown(description="",layout=widgets.Layout(width="35%"), padding=4, options=[""])
    text16 = widgets.Dropdown(description="",layout=widgets.Layout(width="35%"), padding=4, options=[""])
    b5 = widgets.Button(description="Show Statistics", layout=widgets.Layout(width="10%"))
    b5.style.button_color = "lightgray"

    file = open("datasets3.png", "rb")
    image = file.read()
    img = widgets.Image(value=image,format='png',width=250,height=350)

    h14 = widgets.VBox([widgets.HBox([l1,spc,text14]),spc,widgets.HBox([l2,spc,text15]),spc,widgets.HBox([l3,spc,text16,spc,b5]),spc,img])

    html1 = "<h1 style='font-size:15px'> MRR  </h1>"
    html2 = "<h1 style='font-size:15px'> Similar Objects </h1>"

    l4 = widgets.Label("Distance")
    text17 = widgets.Dropdown(description='',layout=widgets.Layout(width="35%"),options=["Edit Distance","DTW"])

    Loc = widgets.Checkbox(value=False,description='Location',disabled=False,indent=False)
    Traj = widgets.Checkbox(value=False,description='Trajectory',disabled=False,indent=False)

    l5 = widgets.Label("Object")
    text18 = widgets.Dropdown(description='',options=[""])

    text_k = widgets.Label("Top-k")
    text19 = widgets.Text(description='',layout=widgets.Layout(width="25%"))

    file = open("mrr_trajs2.png", "rb")
    image = file.read()
    img1 = widgets.Image(value=image,format='png',width=600,height=700)

    #file = open("índicep.jpeg", "rb")
    #image = file.read()
    #img2 = widgets.Image(value=image,format='png',width=400,height=500)

    b6 = widgets.Button(description="Plot", layout=widgets.Layout(width="30%"))
    b6.style.button_color = "lightgray"

    h15 = widgets.VBox([widgets.HBox([l4,spc,text17]),spc,widgets.HBox([Loc,Traj]),spc,widgets.HBox([widgets.HBox([l5,spc,text18]),spc,spc,widgets.HBox([text_k,spc,text19,spc,spc,b6])]),spc,
                                                                 widgets.HBox([widgets.VBox([widgets.HTML(value=html1),img1]),
                                                                               spc,widgets.VBox([widgets.HTML(value=html2)])])])
  
  
    display(widgets.VBox([h14,spc,spc,h15]))


# Trajectory Data / Trajectory Embedding

# TrajEmb Upload: Embedding Importado / 
# Visualize: Em vez janela de texto um dataframe; Calculate = Show Statistics; View Trajectory Data

# Extrinsic Eval: Trajectory Dataset (select); Embedding Model; Dataframe com as estatisticas : Comparativo entre Embediing 
# Similar Object/ MRR Plot
# Location vs Trajectory, Edit e DTW

# 1º aba: Intrinsic Evaluation - Matheus 
# 2º aba: Extrinsic Evaluation - Wilken

# colocar uma label ao subir o arquivo
# Botar código no github
